In [ ]:
# https://www.kaggle.com/datasets/prapannadhungel/nepali-currency-dataset

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-03-11 22:59:04.909270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741714144.936666   98194 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741714144.944201   98194 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 22:59:04.974536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,  # Rotate images slightly
    width_shift_range=0.2,  # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,  # Shear effect
    zoom_range=0.2,  # Zoom into images
    horizontal_flip=True  # Flip images horizontally
)


In [3]:
train_generator = train_datagen.flow_from_directory(
    'Dataset',  # Path to dataset
    target_size=(224, 224),  # Ensure input matches model shape
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Dataset',  # Path to dataset
    target_size=(224, 224),  # Ensure input matches model shape
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 4713 images belonging to 4 classes.
Found 1177 images belonging to 4 classes.


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step 4: Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile the model for multi-class classification
model.compile(
    loss='sparse_categorical_crossentropy',  # Change from binary to categorical loss
    optimizer='adam',
    metrics=['accuracy']
)


train_steps_per_epoch = int(np.ceil(train_generator.samples / train_generator.batch_size))
validation_steps_per_epoch = int(np.ceil(validation_generator.samples / validation_generator.batch_size))

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_steps_per_epoch,
    verbose=1
)


/home/jonah/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1741714149.506914   98194 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2784 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
/home/jonah/.local/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


I0000 00:00:1741714155.512267   98291 service.cc:148] XLA service 0x75c4cc0069d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741714155.512307   98291 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-03-11 22:59:15.562756: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741714155.867849   98291 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-11 22:59:16.585126: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,222,222]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,224,224]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"act

  1/148 ━━━━━━━━━━━━━━━━━━━━ 25:30 10s/step - accuracy: 0.1250 - loss: 1.4048

I0000 00:00:1741714162.731344   98291 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 86/148 ━━━━━━━━━━━━━━━━━━━━ 50s 812ms/step - accuracy: 0.5237 - loss: 1.4177

2025-03-11 23:00:32.530712: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[9,32,222,222]{3,2,1,0}, u8[0]{0}) custom-call(f32[9,3,224,224]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-03-11 23:00:32.702780: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[9,64,109,109]{3,2,1,0}, u8[0]{0}) custom-call(f32[9,32,111,111]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 837ms/step - accuracy: 0.5710 - loss: 1.1900

2025-03-11 23:01:28.633264: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,222,222]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,224,224]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-03-11 23:01:28.751633: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,109,109]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,111,111]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend

148/148 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.5716 - loss: 1.1876 - val_accuracy: 0.5930 - val_loss: 0.8120
Epoch 2/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 146s 984ms/step - accuracy: 0.7890 - loss: 0.5404 - val_accuracy: 0.7867 - val_loss: 0.6322
Epoch 3/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 144s 972ms/step - accuracy: 0.8815 - loss: 0.3781 - val_accuracy: 0.7647 - val_loss: 0.6486
Epoch 4/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 135s 911ms/step - accuracy: 0.8888 - loss: 0.3062 - val_accuracy: 0.8275 - val_loss: 0.6017
Epoch 5/5
148/148 ━━━━━━━━━━━━━━━━━━━━ 144s 974ms/step - accuracy: 0.9368 - loss: 0.1835 - val_accuracy: 0.8428 - val_loss: 0.5618


In [5]:
# Save the model
model.save('check.h5')


In [6]:
print(model.input_shape)


(None, 224, 224, 3)


In [17]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

# Load the trained model
model_path = 'check.h5'  # Ensure the model file exists
if not os.path.exists(model_path):
    print(f"❌ Error: Model file '{model_path}' not found.")
    exit(1)

try:
    model = load_model(model_path)
    print("✅ Model loaded successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit(1)

# Function to preprocess and predict currency authenticity
def predict_currency(image_path):
    """ Reads an image file, preprocesses it, and predicts if it's REAL or FAKE. """
    try:
        if not os.path.exists(image_path):
            return f"❌ Error: File '{image_path}' does not exist."

        # Load and preprocess the image
        img = Image.open(image_path).convert('RGB')  # Ensure 3 color channels
        img = img.resize((224, 224))  # Resize to match model input size
        img = img_to_array(img) / 255.0  # Normalize pixel values
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img)[0][0]  # Extract prediction value
        confidence = prediction * 100  # Convert to percentage
        label = "REAL" if prediction > 0.5 else "FAKE"

        return f"🔹 Prediction: {label} ({confidence:.2f}% confidence)"
    
    except Exception as e:
        return f"❌ Error processing image: {e}"


# Ask for an image file path
image_path = input("Enter the image file path: ").strip()

if image_path:
    result = predict_currency(image_path)
    print(result)
else:
    print("❌ No file path provided.")


✅ Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
🔹 Prediction: FAKE (8.69% confidence)
